In [6]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd
import re

# --- 1. 定义文件路径 ---
homology_excel_file = "同源表_41588_2023_1620_MOESM4_ESM.xlsx"
homology_sheet_name = "human vs mouse"
human_fasta_file = "gencode.v25.lncRNA_transcripts.fa"
mouse_fasta_file = "gencode.vM10.lncRNA_transcripts.fa"
output_human_fasta = "human_homology_lncRNAs.fa"
output_mouse_fasta = "mouse_homology_lncRNAs.fa"
output_summary_file = "homology_pairs_summary.txt"

# --- 2. 读取同源表Excel文件 ---
print("正在读取同源表Excel文件...")
try:
    homology_df = pd.read_excel(homology_excel_file, sheet_name=homology_sheet_name)
    
    print("同源表DataFrame列名:", homology_df.columns.tolist())
    print("前5行数据:")
    print(homology_df.head())
    
    human_gene_col = homology_df.columns[0]  # 第一列
    mouse_gene_col = homology_df.columns[1]  # 第二列
    human_transcript_col = homology_df.columns[2]  # 第三列
    mouse_transcript_col = homology_df.columns[3]  # 第四列
    
    print(f"使用列: 人类基因 - '{human_gene_col}', 小鼠基因 - '{mouse_gene_col}'")
    print(f"人类转录本 - '{human_transcript_col}', 小鼠转录本 - '{mouse_transcript_col}'")
    
    # 提取同源对信息
    homology_pairs = []
    for index, row in homology_df.iterrows():
        human_gene = str(row[human_gene_col]).strip()
        mouse_gene = str(row[mouse_gene_col]).strip()
        human_transcript = str(row[human_transcript_col]).strip()
        mouse_transcript = str(row[mouse_transcript_col]).strip()
        
        # 跳过空值或无效值
        if (human_gene != 'nan' and mouse_gene != 'nan' and 
            human_transcript != 'nan' and mouse_transcript != 'nan' and
            human_gene and mouse_gene and human_transcript and mouse_transcript):
            
            # 提取得分信息（从第5列开始到最后一列）
            scores = []
            for col in homology_df.columns[4:]:
                score_val = row[col]
                if pd.isna(score_val):
                    scores.append("NA")
                else:
                    scores.append(str(score_val))
            
            homology_pairs.append({
                'human_gene': human_gene,
                'mouse_gene': mouse_gene,
                'human_transcript': human_transcript,  # 无版本号
                'mouse_transcript': mouse_transcript,  # 无版本号
                'scores': scores
            })
    
    print(f"共读取 {len(homology_pairs)} 对有效同源基因对。")
    
except Exception as e:
    print(f"读取同源表Excel文件时出错: {e}")
    exit()

# --- 3. 构建转录本映射（无版本号到完整ID） ---
def build_transcript_mapping(fasta_file):
    """构建无版本号转录本ID到完整ID的映射"""
    transcript_map = {}
    
    print(f"正在分析 {fasta_file} 中的转录本...")
    
    for record in SeqIO.parse(fasta_file, "fasta"):
        header = record.id
        
        # 从header中提取转录本ID（去除版本号）
        # GENCODE格式: ENSTXXXXX.X|ENSGXXXXX.X|...
        parts = header.split('|')
        if len(parts) >= 1:
            transcript_id_with_version = parts[0]
            transcript_id = transcript_id_with_version.split('.')[0]  # 去除版本号
            
            # 如果同一个无版本号ID对应多个转录本，选择最长的
            if transcript_id not in transcript_map:
                transcript_map[transcript_id] = (header, len(record.seq))
            else:
                # 如果已有映射，比较长度，保留更长的
                current_length = transcript_map[transcript_id][1]
                if len(record.seq) > current_length:
                    transcript_map[transcript_id] = (header, len(record.seq))
    
    print(f"为 {len(transcript_map)} 个转录本构建了映射")
    return transcript_map

# 为人类和小鼠分别构建转录本映射
human_transcript_map = build_transcript_mapping(human_fasta_file)
mouse_transcript_map = build_transcript_mapping(mouse_fasta_file)

# --- 4. 索引FASTA文件 ---
print("正在索引人类FASTA文件...")
human_idx = SeqIO.index(human_fasta_file, "fasta")
print("正在索引小鼠FASTA文件...")
mouse_idx = SeqIO.index(mouse_fasta_file, "fasta")

# --- 5. 提取序列并写入新文件 ---
def write_single_line_fasta(record, output_handle, extra_info=""):
    """将序列写入为单行FASTA格式，可添加额外信息"""
    if extra_info:
        header = f">{record.id}|{extra_info}"
    else:
        header = f">{record.id}"
    output_handle.write(f"{header}\n")
    output_handle.write(f"{str(record.seq)}\n")

# 创建汇总信息
summary_lines = []

with open(output_human_fasta, 'w') as human_out, open(output_mouse_fasta, 'w') as mouse_out:
    human_count = 0
    mouse_count = 0
    processed_pairs = 0
    found_pairs = 0

    # 遍历每一对同源基因
    for pair in homology_pairs:
        human_transcript_base = pair['human_transcript']  # 无版本号
        mouse_transcript_base = pair['mouse_transcript']  # 无版本号
        scores_str = "|".join(pair['scores'])
        
        found_human = False
        found_mouse = False
        
        # 获取人类转录本序列
        if human_transcript_base in human_transcript_map:
            transcript_id_with_version = human_transcript_map[human_transcript_base][0]
            if transcript_id_with_version in human_idx:
                seq_record = human_idx[transcript_id_with_version]
                # 在FASTA头中添加小鼠转录本信息和得分信息
                extra_info = f"{pair['mouse_transcript']}|{scores_str}"
                write_single_line_fasta(seq_record, human_out, extra_info)
                found_human = True
                human_count += 1
        
        # 获取小鼠转录本序列
        if mouse_transcript_base in mouse_transcript_map:
            transcript_id_with_version = mouse_transcript_map[mouse_transcript_base][0]
            if transcript_id_with_version in mouse_idx:
                seq_record = mouse_idx[transcript_id_with_version]
                # 在FASTA头中添加人类转录本信息和得分信息
                extra_info = f"{pair['human_transcript']}|{scores_str}"
                write_single_line_fasta(seq_record, mouse_out, extra_info)
                found_mouse = True
                mouse_count += 1
        
        # 记录汇总信息
        status = "成功" if found_human and found_mouse else "部分成功" if found_human or found_mouse else "失败"
        summary_lines.append(f"{pair['human_gene']}\t{pair['mouse_gene']}\t{pair['human_transcript']}\t{pair['mouse_transcript']}\t{status}")
        
        # 统计找到的配对
        if found_human and found_mouse:
            found_pairs += 1
        
        processed_pairs += 1
        if processed_pairs % 1000 == 0:
            print(f"已处理 {processed_pairs}/{len(homology_pairs)} 对同源基因...")
            print(f"  找到 {found_pairs} 对完全匹配的基因")

print("\n完成！")
print(f"从人类文件中提取了 {human_count} 条序列到 {output_human_fasta}")
print(f"从小鼠文件中提取了 {mouse_count} 条序列到 {output_mouse_fasta}")
print(f"成功找到 {found_pairs}/{len(homology_pairs)} 对完全匹配的基因")

# 保存汇总信息
with open(output_summary_file, 'w') as summary_out:
    summary_out.write("人类基因\t小鼠基因\t人类转录本\t小鼠转录本\t提取状态\n")
    for line in summary_lines:
        summary_out.write(f"{line}\n")

# 关闭索引以释放资源
human_idx.close()
mouse_idx.close()

# 显示一些统计信息
print("\n=== 统计信息 ===")
print(f"总同源对: {len(homology_pairs)}")
print(f"成功提取的人类序列: {human_count}")
print(f"成功提取的小鼠序列: {mouse_count}")
print(f"汇总信息已保存到: {output_summary_file}")

# # 可选：保存转录本映射关系
# with open("human_transcript_mapping.txt", 'w') as f:
#     for base_id, (full_id, length) in human_transcript_map.items():
#         f.write(f"{base_id}\t{full_id}\t{length}\n")

# with open("mouse_transcript_mapping.txt", 'w') as f:
#     for base_id, (full_id, length) in mouse_transcript_map.items():
#         f.write(f"{base_id}\t{full_id}\t{length}\n")

正在读取同源表Excel文件...
同源表DataFrame列名: ['human gene', 'mouse gene', 'human transcript', 'mouse transcript', 'with homologs in cow', 'with homologs in opossum', 'with homologs in chicken', 'with homologs in lizard', 'with homologs in frog', 'with homologs in zebrafish', 'motif score ', 'gap score']
前5行数据:
        human gene          mouse gene human transcript    mouse transcript  \
0  ENSG00000224511  ENSMUSG00000085237  ENST00000413591  ENSMUST00000133694   
1  ENSG00000230498  ENSMUSG00000105337  ENST00000454305  ENSMUST00000198047   
2  ENSG00000279996  ENSMUSG00000106758  ENST00000623029  ENSMUST00000201839   
3  ENSG00000270959  ENSMUSG00000097867  ENST00000605573  ENSMUST00000181640   
4  ENSG00000248050  ENSMUSG00000097102  ENST00000499579  ENSMUST00000181839   

   with homologs in cow  with homologs in opossum  with homologs in chicken  \
0                     0                         0                         0   
1                     1                         0                 